In [1]:
import pandas as pd
canaux_nf = pd.read_csv('3_canaux_non_fusionnees.csv', delimiter=',', index_col=0)
canaux_vf = pd.read_csv('3_canaux_vraiment_fusionnees.csv', delimiter=',', index_col=0)
canaux = pd.read_csv('1_canaux.csv',delimiter=',',index_col=0)

In [2]:
import folium
import pandas as pd
import pyproj
import json
from branca.element import Template, MacroElement
import geopandas as gpd


# Convertir les coordonnées Lambert II étendu en coordonnées latitude/longitude
lambert = pyproj.Proj("+init=EPSG:27572")
wgs84 = pyproj.Proj("+init=EPSG:4326")

# Convertir les coordonnées des canaux en latitude/longitude
A_canaux = canaux[['X1', 'Y1']].values
B_canaux = canaux[['X2', 'Y2']].values
A_canaux_nf = canaux_nf[['X1', 'Y1']].values
B_canaux_nf = canaux_nf[['X2', 'Y2']].values
A_canaux_vf = canaux_vf[['X1', 'Y1']].values
B_canaux_vf = canaux_vf[['X2', 'Y2']].values

#On transforme les coordonnées en wgs84
A_canaux_latlon = pyproj.transform(lambert, wgs84, A_canaux[:, 0], A_canaux[:, 1])
B_canaux_latlon = pyproj.transform(lambert, wgs84, B_canaux[:, 0], B_canaux[:, 1])
A_canaux_nf_latlon = pyproj.transform(lambert, wgs84, A_canaux_nf[:, 0], A_canaux_nf[:, 1])
B_canaux_nf_latlon = pyproj.transform(lambert, wgs84, B_canaux_nf[:, 0], B_canaux_nf[:, 1])
A_canaux_vf_latlon = pyproj.transform(lambert, wgs84, A_canaux_vf[:, 0], A_canaux_vf[:, 1])
B_canaux_vf_latlon = pyproj.transform(lambert, wgs84, B_canaux_vf[:, 0], B_canaux_vf[:, 1])

# Créer une carte centrée sur la Bretagne
m = folium.Map(location=[48.495722, -1.578333], zoom_start=12)

# Créer une couche pour les canaux
canaux_folium = folium.FeatureGroup(name='canaux_folium')
m.add_child(canaux_folium)
# Ajouter les segments des canaux à la couche canaux_folium avec les extrémités
for i in range(len(canaux)):
    start = [A_canaux_latlon[1][i], A_canaux_latlon[0][i]]
    end = [B_canaux_latlon[1][i], B_canaux_latlon[0][i]]
    folium.PolyLine(locations=[start, end], color='blue').add_to(canaux_folium)
    # Ajouter un marqueur pour l'extrémité de départ
    folium.CircleMarker(location=start, radius=6, color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(canaux_folium)
    folium.CircleMarker(location=end, radius=6, color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(canaux_folium)

#Créer une couche pour les canaux non fusionnées
canaux_nf_folium = folium.FeatureGroup(name='canaux_nf_folium')
m.add_child(canaux_nf_folium)
#Ajouter les segments des canaux non fusionnées à la couche canaux_nf_folium avec les extrémités
for i in range(len(canaux_nf)):
    start = [A_canaux_nf_latlon[1][i], A_canaux_nf_latlon[0][i]]
    end = [B_canaux_nf_latlon[1][i], B_canaux_nf_latlon[0][i]]
    folium.PolyLine(locations=[start, end], color='darkblue').add_to(canaux_nf_folium)
    # Ajouter un marqueur pour l'extrémité de départ
    folium.CircleMarker(location=start, radius=4, color='darkblue', fill=True, fill_color='darkblue', fill_opacity=0.5).add_to(canaux_nf_folium)
    folium.CircleMarker(location=end, radius=4, color='darkblue', fill=True, fill_color='darkblue', fill_opacity=0.5).add_to(canaux_nf_folium)

#Créer une couche pour les canaux vraiment fusionnées
canaux_vf_folium = folium.FeatureGroup(name='canaux_vf_folium')
m.add_child(canaux_vf_folium)
#Ajouter les segments des canaux vraiment fusionnées à la couche canaux_vf_folium avec les extrémités
for i in range(len(canaux_vf)):
    start = [A_canaux_vf_latlon[1][i], A_canaux_vf_latlon[0][i]]
    end = [B_canaux_vf_latlon[1][i], B_canaux_vf_latlon[0][i]]
    folium.PolyLine(locations=[start, end], color='cyan').add_to(canaux_vf_folium)
    # Ajouter un marqueur pour l'extrémité de départ
    folium.CircleMarker(location=start, radius=4, color='cyan', fill=True, fill_color='cyan', fill_opacity=0.5).add_to(canaux_vf_folium)
    folium.CircleMarker(location=end, radius=4, color='cyan', fill=True, fill_color='cyan', fill_opacity=0.5).add_to(canaux_vf_folium)
        
# Ajouter les couches à la carte
folium.LayerControl().add_to(m)

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Carte des canaux avant/après fusion</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">
  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});
  </script>
</head>

#Corps du fichier html
<body>
 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; left: 80px; top: 50px;'>
<div class='legend-title'>Visualisation des Linéaires de Bretagne </div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:blue;opacity:0.7;'></span>Canaux</li>
    <li><span style='background:darkblue;opacity:0.7;'></span>Canaux non fusionnées</li>
    <li><span style='background:cyan;opacity:0.7;'></span>Canaux vraiment fusionnées</li>
  </ul>
</div>
</div>
 
</body>
</html>
<style type='text/css'>
  .maplegend .legend-title {
    text-align: center;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
m.get_root().add_child(macro)

# Enregistrer la carte
m.save('4_carte_fusion_canaux.html')

c:\Users\cypri\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\cypri\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\cypri\AppData\Local\Temp\ipykernel_8072\1964019225.py:22: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotc